In [1]:
%matplotlib inline
import os
from glob import glob
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from PIL import Image
# import keras
# import keras.backend as K
# from skimage.util.montage import montage2d
# from skimage.io import imread
# from scipy.io import loadmat # for loading mat files
# from tqdm import tqdm_notebook
# root_mpi_dir = os.path.join('..', 'data', 'MPII')
# data_dir = os.path.join(root_mpi_dir, 'Data')
# annot_dir = os.path.join(root_mpi_dir, 'Annotation Subset') # annotations the important part of the data
# img_dir = os.path.join(data_dir, 'Original')

import torch
import torchvision
from torch.utils import data
from torch.utils.data import Dataset
import os
from os import listdir
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import scipy.io as sio
from os import listdir
from os.path import isfile, join
import json

from pathlib import Path

In [2]:
index_to_label = {
    0: 'rowing',
    1: 'badminton',
    2: 'polo',
    3: 'bocce',
    4: 'snowboarding',
    5: 'croquet',
    6: 'sailing',
    7: 'rockclimbing',
}

In [3]:
label_to_index = {v: k for k, v in index_to_label.items()}

In [4]:
# mpii_path = '../data/MPII'
images_path = os.path.join('../data/')

In [5]:
image_files = [f for f in listdir(images_path) if isfile(join(images_path, f))]


## Set up Data

In [6]:
image_dict = {}
label_dict = {}

index = 0
# for i in range(len(image_files)):
for i in range(100):
    im = image_files[i]
    if 'Thumb' in im or '.DS' in im:
        continue
    if 'rar' in im:
        continue
#     print(im)
    label_str = im.split('_')[2].lower()
    image_dict[index] = im
    label_dict[index] = label_to_index[label_str]
    index += 1

In [7]:
class UIUC_Actions_Dataset(data.Dataset):
#       '''Characterizes a dataset for PyTorch'''
    def __init__(self, labels, images, images_path):
        '''Initialization'''
        self.labels = labels
        self.images = images
        
        self.images_path = images_path
        
        self.transform = transforms.Compose(
                [
                    transforms.Resize((96, 96)),
                    transforms.ToTensor(),
#                     transforms.CenterCrop(10),
                 
                 transforms.Normalize((0.5, 0.5, 0.5), 
                                      (0.5, 0.5, 0.5))])

    def __len__(self):
        '''Denotes the total number of samples'''
        return len(self.labels)

    def __getitem__(self, index):
        '''Generates one sample of data'''
        # Select sample
        image_filename = self.images[index]
        path_to_image = os.path.join(self.images_path, image_filename)

        # Load data and get label
        image = Image.open(path_to_image)
        image = self.transform(image).float()
        x = image
#         y = torch.tensor(np.array(self.labels[index])).float()

#         print(y)
        
        y = int(self.labels[index])

        return x, y

## Make Dataset

In [8]:
train_dataset = UIUC_Actions_Dataset(label_dict, 
                              image_dict, images_path)

train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=256,
                                          shuffle=True,
                                         )


In [9]:
print("len(train_dataset) = ", len(train_dataset))

len(train_dataset) =  100


In [12]:
from openpose import pyopenpose as op

ModuleNotFoundError: No module named 'openpose'